# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df_1 = data.groupby(['CustomerID','ProductName']).sum()
df_1 = df_1.drop(columns=['SalesID','ProductID'])
df_1

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
df_2 = pd.pivot_table(df_1, index='ProductName')
df_2 = pd.pivot_table(df_1, values='Quantity', index=['ProductName'], columns=['CustomerID'],fill_value=0)
df_2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [30]:
distances = pd.DataFrame(1/(1 + squareform(pdist(df_2.T, 'euclidean'))), 
                         index=df_2.columns, columns=df_2.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similarities = distances[33].sort_values(ascending=False)[1:6]
parecidos = list(similarities.index)
parecidos

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
step5 = df_1.loc[parecidos]
step5

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
df_6 = step5.groupby(['ProductName']).sum()
df_6 = df_6.sort_values(by='Quantity',ascending=False)
df_6

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
customer = df_1.loc[33]
customer

,Quantity
ProductName,
Apricots - Dried,1
Assorted Desserts,1
Bandage - Flexible Neon,1
"Bar Mix - Pina Colada, 355 Ml",1
"Beans - Kidney, Canned",1
"Beef - Chuck, Boneless",1
Beef - Prime Rib Aaa,1
Beer - Original Organic Lager,1
Beer - Rickards Red,1


In [11]:
result = pd.concat([df_6, customer], axis=1,sort=False).fillna(0.0)
result.columns = ['Quantity Similar','Quantity Customer']
result = result[result['Quantity Customer'] == 0.0]
result = result.sort_values(by='Quantity Similar' ,ascending=False).head(5).reset_index()
result

,index,Quantity Similar,Quantity Customer
0,Butter - Unsalted,3.0,0.0
1,Soup - Campbells Bean Medley,3.0,0.0
2,Wine - Blue Nun Qualitatswein,3.0,0.0
3,Wine - Ej Gallo Sierra Valley,3.0,0.0
4,Beans - Wax,2.0,0.0


In [12]:
result = list(result['index'])
result

['Butter - Unsalted',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Wine - Ej Gallo Sierra Valley',
 'Beans - Wax']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [17]:
customers_ids = list(set(data['CustomerID']))

In [20]:
recomen = {}
for i in customers_ids: 
    similarities = distances[i].sort_values(ascending=False)[1:6]
    parecidos = list(similarities.index)
    step5 = df_1.loc[parecidos]
    df_6 = step5.groupby(['ProductName']).sum()
    df_6 = df_6.sort_values(by='Quantity',ascending=False)
    customer = df_1.loc[i]
    result = pd.concat([df_6, customer], axis=1,sort=False).fillna(0.0)
    result.columns = ['Quantity Similar','Quantity Customer']
    result = result[result['Quantity Customer'] == 0.0]
    result = result.sort_values(by='Quantity Similar' ,ascending=False).head(5).reset_index()
    result = list(result['index'])
    recomen[i] = result

In [21]:
recomen

{83973: ['Cocoa Butter',
  'Veal - Slab Bacon',
  'Bandage - Fexible 1x3',
  'Ezy Change Mophandle',
  'Chicken - Wieners'],
 59399: ['Shrimp - Baby, Warm Water',
  'Towels - Paper / Kraft',
  'Knife Plastic - White',
  'Nantucket - Pomegranate Pear',
  'Snapple - Iced Tea Peach'],
 92168: ['Cup - Translucent 7 Oz Clear',
  'Snapple Lemon Tea',
  'Veal - Brisket, Provimi,bnls',
  'Spinach - Baby',
  'Pop Shoppe Cream Soda'],
 49159: ['Pasta - Penne, Rigate, Dry',
  'Cookies - Assorted',
  'Puree - Mocha',
  'Cheese - Parmesan Grated',
  'Cheese - Victor Et Berthold'],
 18441: ['Soupfoamcont12oz 112con',
  'Beef - Texas Style Burger',
  'Anchovy Paste - 56 G Tube',
  'Flour - Pastry',
  'Foam Cup 6 Oz'],
 22536: ['Bacardi Breezer - Tropical',
  'Cheese - Mozzarella',
  'Cheese Cloth No 100',
  'Bread - Hot Dog Buns',
  'Table Cloth - 53x69 Colour'],
 86028: ['Spinach - Baby',
  'Vinegar - Tarragon',
  'Pork - Hock And Feet Attached',
  'Orange - Canned, Mandarin',
  'Pepper - Black, Who

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [26]:
df_9 = pd.DataFrame(recomen)
df_9

,83973,59399,92168,49159,18441,22536,86028,75791,96272,32785,...,55281,75762,45042,59379,77810,77814,32753,20476,67582,65535
0,Cocoa Butter,"Shrimp - Baby, Warm Water",Cup - Translucent 7 Oz Clear,"Pasta - Penne, Rigate, Dry",Soupfoamcont12oz 112con,Bacardi Breezer - Tropical,Spinach - Baby,Blueberries,Ecolab - Lime - A - Way 4/4 L,Cookies - Assorted,...,V8 - Berry Blend,Ice Cream Bar - Oreo Cone,Sea Bass - Whole,Cod - Black Whole Fillet,Olive - Spread Tapenade,Cheese - Victor Et Berthold,Grouper - Fresh,Soup - Campbells Tomato Ravioli,Cake - Mini Cheesecake,Sauce - Rosee
1,Veal - Slab Bacon,Towels - Paper / Kraft,Snapple Lemon Tea,Cookies - Assorted,Beef - Texas Style Burger,Cheese - Mozzarella,Vinegar - Tarragon,Vaccum Bag 10x13,Guinea Fowl,Juice - V8 Splash,...,"Pepper - Paprika, Hungarian",Vol Au Vents,Ice Cream Bar - Oreo Cone,"Pork - Back, Short Cut, Boneless",Pants Custom Dry Clean,Macaroons - Two Bite Choc,Sherry - Dry,"Pasta - Detalini, White, Fresh",Ecolab - Solid Fusion,"Chestnuts - Whole,canned"
2,Bandage - Fexible 1x3,Knife Plastic - White,"Veal - Brisket, Provimi,bnls",Puree - Mocha,Anchovy Paste - 56 G Tube,Cheese Cloth No 100,Pork - Hock And Feet Attached,Turkey - Oven Roast Breast,Bread Crumbs - Panko,Wine - Redchard Merritt,...,Soupcontfoam16oz 116con,Soupcontfoam16oz 116con,Spinach - Baby,Thermometer Digital,Sword Pick Asst,Tilapia - Fillets,"Chestnuts - Whole,canned",Skirt - 29 Foot,Wine - Redchard Merritt,Scallop - St. Jaques
3,Ezy Change Mophandle,Nantucket - Pomegranate Pear,Spinach - Baby,Cheese - Parmesan Grated,Flour - Pastry,Bread - Hot Dog Buns,"Orange - Canned, Mandarin",Mussels - Cultivated,Sunflower Seed Raw,Puree - Passion Fruit,...,Brandy - Bar,Ketchup - Tomato,Beets - Mini Golden,Beans - Wax,Arizona - Green Tea,Wine - Ruffino Chianti,Napkin White - Starched,Wanton Wrap,Mussels - Frozen,"Bread - Roll, Soft White Round"
4,Chicken - Wieners,Snapple - Iced Tea Peach,Pop Shoppe Cream Soda,Cheese - Victor Et Berthold,Foam Cup 6 Oz,Table Cloth - 53x69 Colour,"Pepper - Black, Whole",Bread - Calabrese Baguette,Bread - English Muffin,Juice - Orange,...,Vinegar - Tarragon,Anchovy Paste - 56 G Tube,Wine - Redchard Merritt,Sponge Cake Mix - Chocolate,Pail For Lid 1537,Cake - Cake Sheet Macaroon,Sprouts - Alfalfa,Broom - Corn,Cheese - Bocconcini,Chicken - Wieners


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [29]:
distances_1 = pd.DataFrame(1/(1 + squareform(pdist(df_2.T, 'correlation'))), 
                         index=df_2.columns, columns=df_2.columns)
distances_1

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.493776,0.529677,0.510360,0.506951,0.507217,0.487410,0.497932,0.499215,0.502499,...,0.493385,0.484376,0.493486,0.479867,0.499870,0.479764,0.493197,0.486457,0.491384,0.492072
200,0.493776,1.000000,0.499733,0.498575,0.482172,0.481921,0.519139,0.480599,0.499131,0.505459,...,0.504719,0.496170,0.517884,0.508954,0.492581,0.484514,0.534491,0.500253,0.484272,0.488735
264,0.529677,0.499733,1.000000,0.497579,0.513573,0.490305,0.503243,0.480936,0.502028,0.490242,...,0.508803,0.490805,0.499750,0.499162,0.517743,0.525278,0.511845,0.503500,0.515736,0.490875
356,0.510360,0.498575,0.497579,1.000000,0.490143,0.489778,0.501933,0.507803,0.482686,0.486354,...,0.499105,0.486660,0.498662,0.502228,0.494239,0.496086,0.484207,0.502204,0.506401,0.500631
412,0.506951,0.482172,0.513573,0.490143,1.000000,0.497577,0.505111,0.512470,0.493947,0.507777,...,0.488109,0.529502,0.492033,0.495264,0.504752,0.500064,0.481259,0.502418,0.505997,0.515015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.479764,0.484514,0.525278,0.496086,0.500064,0.507963,0.504739,0.499300,0.500268,0.492619,...,0.494577,0.504692,0.487604,0.510055,0.534474,1.000000,0.525583,0.508923,0.492050,0.482921
98069,0.493197,0.534491,0.511845,0.484207,0.481259,0.484219,0.496892,0.490306,0.495330,0.506038,...,0.489353,0.499742,0.493332,0.484621,0.518025,0.525583,1.000000,0.511707,0.501063,0.498897
98159,0.486457,0.500253,0.503500,0.502204,0.502418,0.499195,0.500240,0.502118,0.510333,0.502173,...,0.505074,0.522726,0.500553,0.489177,0.506341,0.508923,0.511707,1.000000,0.504367,0.495580


In [31]:
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266
